In [1]:
from itertools import chain, combinations
import copy
import itertools
import pandas as pd
import math

In [2]:
def reorder_links(user_stateA,user_stateB):
    if len(user_stateA) <= 1 and len(user_stateB) >= 1:
        
        temp_user_stateA = copy.deepcopy(user_stateA)
        temp_user_stateB = copy.deepcopy(user_stateB)
        
        user_stateA = copy.deepcopy(temp_user_stateB)
        user_stateB = copy.deepcopy(temp_user_stateA)
    
    if len(user_stateA) == 1 and len(user_stateB) == 1:
        if user_stateA[0] < user_stateB[0]:
            temp2user_stateA = copy.deepcopy(user_stateA)
            temp2user_stateB = copy.deepcopy(user_stateB)
            
            user_stateA = copy.deepcopy(temp2user_stateB)
            user_stateB = copy.deepcopy(temp2user_stateA)
        elif user_stateA[0] == None:
            temp2user_stateA = copy.deepcopy(user_stateA)
            temp2user_stateB = copy.deepcopy(user_stateB)
            
            user_stateA = copy.deepcopy(temp2user_stateB)
            user_stateB = copy.deepcopy(temp2user_stateA)
              
    return user_stateA,user_stateB  

In [3]:
def findlinks_by_age(state, age_pref):
    age_type = age_pref.split()[0]
    age_position = int(age_pref.split()[1])

    if age_type == "newest":
        sorted_state = sorted(state)
        age_of_req_link = sorted_state[age_position-1]

        
    elif age_type == "oldest":
        sorted_state = sorted(state, reverse=True)
        age_of_req_link = sorted_state[age_position-1]

            
    index_of_req_link = state.index(age_of_req_link)
    
    return age_of_req_link, index_of_req_link

In [4]:
def distillation_fidelity(link1_age, link2_age):
    F_link1 = math.e**(-1*alpha*link1_age)
    F_link2 = math.e**(-1*alpha*link2_age)
    
    p_succ = ((8/9)*(F_link1*F_link2))-((2/9)*(F_link1 + F_link2)) + (5/9)
    
    resultant_fidelity = (((10/9)*(F_link1*F_link2))-((1/9)*(F_link1 + F_link2)) + (1/9))/(p_succ)
    return resultant_fidelity, p_succ

In [5]:
def ES_fidelity(link1_age, link2_age):
    fidelity = math.e**(-1*alpha*link1_age) * math.e**(-1*alpha*link2_age)
    
    return fidelity

In [6]:
def timeProgression(user_stateA,user_stateB, m_star):
    # We progress ages of all links by 1 and remove links older than m_star age
    user_stateA_new = list(map(lambda x : x + 1, user_stateA))
    user_stateB_new = list(map(lambda x : x + 1, user_stateB))
    
    user_stateA_fin = list(filter(lambda x: x <= m_star, user_stateA_new))
    user_stateB_fin = list(filter(lambda x: x <= m_star, user_stateB_new))
    
    return user_stateA_fin,user_stateB_fin

In [7]:
# If agreed to add negative reward for DN, we will add a conditional statement to check when DN is being done
# and then return negative reward
def DN(user_stateA,user_stateB,breakFlag):
    if len(user_stateA) > 0 and len(user_stateB) > 0:
        return user_stateA, user_stateB, reward, True
    else:   
        return user_stateA, user_stateB, 0, breakFlag


In [8]:
def ES_fail(user_stateA, user_stateB, breakFlag,ES_age_pref_A,ES_age_pref_B):
    
    '''
    Parameters:
    
    ES_age_pref_A :- this variable defines the choice the generated link for distillation from queue A
    ES_age_pref_B :- this variable defines the choice the generated link for distillation from queue A
    '''
    age_position_A = int(ES_age_pref_A.split()[1])
    age_position_B = int(ES_age_pref_B.split()[1])
    reward = 0
    
    # Switch A and B queues if A is length 1 and B is greater than 1
    user_stateA,user_stateB = reorder_links(user_stateA,user_stateB)
    
    # Actual ES_fail logic
    if len(user_stateA) >= age_position_A and len(user_stateB) >= age_position_B :
        link_A_age, chosen_link_index_A = findlinks_by_age(user_stateA, ES_age_pref_A)
        link_B_age, chosen_link_index_B = findlinks_by_age(user_stateB, ES_age_pref_B)
    
        user_stateA.pop(chosen_link_index_A)
        user_stateB.pop(chosen_link_index_B)
        return user_stateA, user_stateB, reward, False
    else:
        return user_stateA, user_stateB, reward, True

In [9]:
def ES_success(user_stateA, user_stateB, breakFlag,ES_age_pref_A,ES_age_pref_B):
    # Here while calculating reward, I have not multiplied by the probability of ES success q because
    # this is just the action and state space wherein ES is confidently succeeding. This is not for creating the Bellman equation.
    # Rather it is only to see how the states are progressing
    '''
    Parameters:
    
    ES_age_pref_A :- this variable defines the choice the generated link for distillation from queue A
    ES_age_pref_B :- this variable defines the choice the generated link for distillation from queue A
    '''
    
    reward = 0
    
    # Switch A and B queues if A is length 1 and B is greater than 1
    user_stateA,user_stateB = reorder_links(user_stateA,user_stateB)
    
    age_position_A = int(ES_age_pref_A.split()[1])
    age_position_B = int(ES_age_pref_B.split()[1])
    
    # Actual ES_succeeds logic
        
    if len(user_stateA) >= age_position_A and len(user_stateB) >= age_position_B:
        link_A_age, chosen_link_index_A = findlinks_by_age(user_stateA, ES_age_pref_A)
        link_B_age, chosen_link_index_B = findlinks_by_age(user_stateB, ES_age_pref_B)
        
        user_stateA.pop(chosen_link_index_A)
        user_stateB.pop(chosen_link_index_B)
        
        ES_fidelity_fin = ES_fidelity(link_A_age,link_B_age)
        
        reward = ES_fidelity_fin
        return user_stateA, user_stateB, reward, False
    else:
        return user_stateA, user_stateB, reward, True


In [10]:
def distill_success(user_stateA, user_stateB, breakFlag,distill_age_pref_A_first, distill_age_pref_A_second, ES_age_pref_B):
    
    '''
    Here too just calculating resultant fidelity due to ES and SoD. Not multiplying the resultant fidelity with
    p_suuc and q like in equation 16
    '''
    
    #distill_age_pref_A_first = "newest 1" 
    #distill_age_pref_A_second = "newest 2"
    
    #ES_age_pref_B = "newest 1"
    
    age_position_A = int(distill_age_pref_A_second.split()[1])
    age_position_B = int(ES_age_pref_B.split()[1])
    
    psucc = 0
    reward = 0
    
    # Switch A and B queues if A is length 1 and B is greater than 1
    user_stateA,user_stateB = reorder_links(user_stateA,user_stateB)
        
    # Actual Distillation Success logic
    if len(user_stateA) >= age_position_A and len(user_stateB) >= age_position_B:
        link_A_age_first, chosen_link_index_A_first = findlinks_by_age(user_stateA, distill_age_pref_A_first)
        link_A_age_second, chosen_link_index_A_second = findlinks_by_age(user_stateA, distill_age_pref_A_second)
        
        link_B_age, chosen_link_index_B = findlinks_by_age(user_stateB, ES_age_pref_B)
        
        user_stateA.pop(chosen_link_index_A_first)
        user_stateA.pop(chosen_link_index_A_second)
        
        user_stateB.pop(chosen_link_index_B)
        
        fidelity__distillationA, psucc = distillation_fidelity(link_A_age_first,link_A_age_second)
        
        reward = fidelity__distillationA * math.e**(-1*alpha*link_B_age)
        
        return user_stateA, user_stateB, reward, False, psucc
    else:
        return user_stateA, user_stateB, reward, True, psucc
    
        

In [11]:
def distill_fail(user_stateA, user_stateB, breakFlag,distill_age_pref_A_first, distill_age_pref_A_second, ES_age_pref_B):
    
    #distill_age_pref_A_first = "newest 1" 
    #distill_age_pref_A_second = "newest 2"
    
    #ES_age_pref_B = "newest 1"
    age_position_A = int(distill_age_pref_A_second.split()[1])
    age_position_B = int(ES_age_pref_B.split()[1])
    reward = 0
    
    if len(user_stateA) >= age_position_A and len(user_stateB) >= age_position_B: #write if condition to reject possibilty of choice links not available
        link_A_age_first, chosen_link_index_A_first = findlinks_by_age(user_stateA, distill_age_pref_A_first)
        link_A_age_second, chosen_link_index_A_second = findlinks_by_age(user_stateA, distill_age_pref_A_second)
        
        link_B_age, chosen_link_index_B = findlinks_by_age(user_stateB, ES_age_pref_B)
        
        user_stateA.pop(chosen_link_index_A_first)
        user_stateA.pop(chosen_link_index_A_second)
        
        user_stateB.pop(chosen_link_index_B)
        
        return user_stateA, user_stateB, reward, False
    else:
        return user_stateA, user_stateB, reward, True
    

In [12]:

def generate_all_states(m_star):
    # see Notes, Points section for assumptions
    age_list = []
    user_stateA = []
    user_stateB = []
    possible_states_Q2 = [[],[0]] # our assumptions restrict the values the second queue can take. ie first of all it cannot accumulate links
    for age in range(0,m_star+1,1):
        age_list.append(age)
        
    state_cands = list(chain.from_iterable(combinations(age_list, r) for r in range(len(age_list)+1)))
    fin_state_cands = []
    for state in state_cands:
        fin_state_cands.append(sorted(list(state),reverse = True))
    
    list_temp = [copy.deepcopy(fin_state_cands), possible_states_Q2]

    combination = [sorted(p,reverse = True) for p in itertools.product(*list_temp)]
    fin_comb = []
    for comb in combination:
        if comb not in fin_comb:
            
            fin_comb.append(comb)
            
    return(fin_comb)


In [13]:
def bothLink_succeed(user_stateA, user_stateB, plink):
    user_stateA_res = copy.deepcopy(user_stateA)
    user_stateB_res = copy.deepcopy(user_stateB)
    
    user_stateA_res.append(0)
    user_stateB_res.append(0)
    return user_stateA_res, user_stateB_res, plink**2

In [14]:
def linkA_succeed(user_stateA, user_stateB, plink):
    user_stateA_res = copy.deepcopy(user_stateA)
    user_stateB_res = copy.deepcopy(user_stateB)
    
    user_stateA_res.append(0)
    return user_stateA_res, user_stateB_res, plink*(1-plink)

In [15]:
def linkB_succeed(user_stateA, user_stateB, plink):
    user_stateA_res = copy.deepcopy(user_stateA)
    user_stateB_res = copy.deepcopy(user_stateB)
    
    user_stateB_res.append(0)
    return user_stateA_res, user_stateB_res, plink*(1-plink)

In [16]:
def bothLink_fail(user_stateA, user_stateB, plink):
    user_stateA_res = copy.deepcopy(user_stateA)
    user_stateB_res = copy.deepcopy(user_stateB)
    
    return user_stateA_res, user_stateB_res, (1-plink)*(1-plink)

In [17]:
output_columns = ["Initial State","Action Taken", "Choice of Links", "Expected Reward", "Actual Reward", "End State(Time progression accounted for)","Intermediate Transition Probability after action","link gen result","Final State","Transition Probability"]


        
output_df = pd.DataFrame(columns=output_columns)
output_df

,Initial State,Action Taken,Choice of Links,Expected Reward,Actual Reward,End State(Time progression accounted for),Intermediate Transition Probability after action,link gen result,Final State,Transition Probability


In [20]:
global alpha 

output_df_series = []
q = 1 #ES success probability # code change required if q not equal to 1
m_star = 2
alpha =  -math.log(0.98)/(m_star)
plink = 0.7 #probability of link generation being successful

####################################choices for distillation list##################################################
choice_distill_ele = [e for e in range(1,m_star+2,1)]

# Using list comprehension + enumerate()
distill_choices = [(a, b) for idx, a in enumerate(choice_distill_ele) for b in choice_distill_ele[idx + 1:]]

##############################################################################################################3333

################################################choices for ES #################################################

ES_choice = [e for e in range(1,m_star+1,1)]

###############################################################################################################3
for e in distill_choices:
    print(e)

(1, 2)
(1, 3)
(2, 3)


In [18]:
actions = [0,1,2]
all_states = generate_all_states(m_star) #add code so that it can run by taking states from excel after each run to progress timestep.
# since failure gives reward 0 and sends to sames state and hence intermediate transition prob is 1, there is no point of taking into account the actions when they fail. actual reward has no consequence
# but when we remove assumptions that lead to the success or failure of actions to change the nexte state, we need to add failure cases too
    
for state in all_states:
    breakFlag = False
    user_stateA = state[0]
    user_stateB = state[1]
    
    
    for action in actions:
        output_list = [state]
        
        
        if action == 0: # Do Nothing
            user_stateA_ch = copy.deepcopy(user_stateA)
            user_stateB_ch = copy.deepcopy(user_stateB)
            user_stateA_act, user_stateB_act, reward, breakFlag = DN(user_stateA_ch,user_stateB_ch,breakFlag) # DN
            
            if breakFlag == False:
                user_stateA_timed, user_stateB_timed = timeProgression(user_stateA_act,user_stateB_act, m_star)
                user_stateA_res = copy.deepcopy(user_stateA_timed)
                user_stateB_res = copy.deepcopy(user_stateB_timed)
                
                expected_reward = reward
                
                output_list.append("DN")
                output_list.append("None")
                output_list.append(expected_reward)
                output_list.append(reward)
                
                output_list.append([user_stateA_res, user_stateB_res])
                output_list.append(1)
                ouput_res = copy.deepcopy(output_list)
                output_df_series.append(ouput_res)
                output_list = [state]
            else:
                continue
        
        elif action == 1: # ES
            
            '''
            We are not considering ES fails option cause it will not change the resultant state. The action is only ES
            using a particular pair of links. If failure would have changed the state to a different one by a probability,
            we would have needed to take that possibility and generate state transition prob vector according to that 
            possibility too to solve MDP. This is because of the assumption that all possible links should be destroyed regardless
            of success or failure. This manipulates the state in the same manner irrespective of failure or success
            '''
            
            for choice in ES_choice:
                user_stateA_ch = copy.deepcopy(user_stateA)
                user_stateB_ch = copy.deepcopy(user_stateB)
                
                user_stateA_act, user_stateB_act, reward, breakFlag = ES_success(user_stateA_ch, user_stateB_ch, breakFlag,"newest " + str(choice),"newest " + str(1))
                
                if breakFlag == False:
                    user_stateA_timed, user_stateB_timed = timeProgression(user_stateA_act,user_stateB_act, m_star)
                    user_stateA_res = copy.deepcopy(user_stateA_timed)
                    user_stateB_res = copy.deepcopy(user_stateB_timed)
                    
                    expected_reward = reward*q  # this is actually summation of all probabilities of getting a particular actual 
                                                # reward * the actual reward. Eg here the value should be reward(succ)*q + reward(fail)(1-q)
                                                # but here reward(fail) = 0 so I just applied reward*q
                    
                    output_list.append("ES")
                    output_list.append(choice)
                    output_list.append(expected_reward)
                    output_list.append(reward)
                    
                    output_list.append([user_stateA_res, user_stateB_res])
                    output_list.append(1) #code to change 
                    ouput_res = copy.deepcopy(output_list)
                    output_df_series.append(ouput_res)
                    output_list = [state]
                else:
                    continue
        
        
        elif action == 2: # Distill 
            '''
            We are not considering distill fails option cause it will not change the resultant state. The action is only distilling and swap
            using a particular set of links. If failure would have changed the state to a different one by a probability,
            we would have needed to take that possibility and generate state transition prob vector according to that 
            possibility too to solve MDP. This is because of the assumption that all possible links should be destroyed regardless
            of success or failure. This manipulates the state in the same manner irrespective of failure or success
            '''
            
            for choice in distill_choices:
                user_stateA_ch = copy.deepcopy(user_stateA)
                user_stateB_ch = copy.deepcopy(user_stateB)
                user_stateA_act, user_stateB_act, reward, breakFlag, psucc = distill_success(user_stateA_ch, user_stateB_ch, breakFlag,"newest " + str(choiceA[0]),"newest " + str(choiceA[1]),"newest " +str(1))
                if breakFlag == False:
                    user_stateA_timed, user_stateB_timed = timeProgression(user_stateA_act,user_stateB_act, m_star)
                    user_stateA_res = copy.deepcopy(user_stateA_timed)
                    user_stateB_res = copy.deepcopy(user_stateB_timed)
                    
                    expected_reward = reward*psucc*q # this is actually summation of all probabilities of getting a particular actual 
                                                # reward * the actual reward. Eg here the value should be reward(succ)*psucc*q + reward(fail)*q*pfail... other possibilities
                                                # but here reward(fail) = 0 for all failure possibilities so I just applied reward*q*psucc
                    
                    output_list.append("Distill")
                    output_list.append(choice)
                    output_list.append(expected_reward)
                    output_list.append(reward)
                    
                    output_list.append([user_stateA_res, user_stateB_res])
                    output_list.append(1) #since distillation fail or success alters the state in the same manner and sends to same state 
                    ouput_res = copy.deepcopy(output_list)
                    output_df_series.append(ouput_res)
                    output_list = [state]
                else:
                    continue
            
        '''
                
        elif action == 4: # Distill Fails
            user_stateA_act, user_stateB_act, reward, breakFlag = distill_fail(user_stateA_ch, user_stateB_ch, breakFlag)
            
            if breakFlag == False:
                user_stateA_timed, user_stateB_timed = timeProgression(user_stateA_act,user_stateB_act, m_star)
                user_stateA_res = copy.deepcopy(user_stateA_timed)
                user_stateB_res = copy.deepcopy(user_stateB_timed)
                
                expected_reward = "Same as Distill Success for this state" #code to change
                
                output_list.append("Distill")
                output_list.append(expected_reward)
                output_list.append(reward)
                
                output_list.append([user_stateA_res, user_stateB_res])
                output_list.append(1) #since distillation fail or success alters the state in the same manner and sends to same state 
            else:
                continue
        '''
         
        '''
        elif action == 2: # ES Fails
            user_stateA_act, user_stateB_act, reward, breakFlag = ES_fail(user_stateA_ch, user_stateB_ch, breakFlag)
            
            if breakFlag == False:
                user_stateA_timed, user_stateB_timed = timeProgression(user_stateA_act,user_stateB_act, m_star)
                user_stateA_res = copy.deepcopy(user_stateA_timed)
                user_stateB_res = copy.deepcopy(user_stateB_timed)
                
                expected_reward = "Same as for ES success this state" # this is actually summation of all probabilities of getting a particular actual 
                                            # reward * the actual reward. Eg here the value should be reward(succ)*q + reward(fail)(1-q)
                                            # but here reward(fail) = 0 so I just applied reward*q
                
                output_list.append("ES")
                output_list.append(expected_reward)
                output_list.append(reward)
                
                output_list.append([user_stateA_res, user_stateB_res])
                output_list.append(1) #code to change   
            else:
                continue
        '''
            

In [19]:

output_df_series_fin = []
link_gen_funcs = [bothLink_succeed, linkA_succeed, linkB_succeed, bothLink_fail]
for func in link_gen_funcs:

    for row in output_df_series:
        row_fin = copy.deepcopy(row)
        link_genA = copy.deepcopy(row_fin[-2][0])
        link_genB = copy.deepcopy(row_fin[-2][1])
        
        int_trans_prob = row_fin[-1]
        user_stateA_fin, user_stateB_fin, transition_prob = func(link_genA, link_genB, plink)
        very_fin_user_stateA, very_fin_user_stateB = reorder_links(user_stateA_fin,user_stateB_fin)
        final_state = [very_fin_user_stateA, very_fin_user_stateB]
        
        row_fin.append(func.__name__)
        row_fin.append(final_state)
        row_fin.append(int_trans_prob*transition_prob)
        
        row_fin_res = copy.deepcopy(row_fin)
        output_df_series_fin.append(row_fin_res)
        

    
    
    

In [20]:
output_df_fin = pd.DataFrame(output_df_series_fin,columns=output_df.columns)

In [21]:
state_space_output = "Final_transition_mStar_"+str(m_star)+"_q_" + str(q) +"_plink_"+str(plink)+"alpha_"+str(alpha)+".xlsx"
output_df_fin.to_excel(state_space_output)